In [6]:
import requests
from bs4 import BeautifulSoup
from lxml import etree
import json
import pandas as pd

from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


<h1 style='text-align:center'> Scarping With Beautiful Soup</h1>

In [7]:
def soup_func(url):
    header = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.0.0'}
    r = requests.get(url, headers=header)
    return BeautifulSoup(r.text, 'lxml')
def pretiffy(soup):
    return soup.prettify()
def xpath(soup, path):
    return etree.HTML(str(soup)).xpath(path)[0].text


In [12]:
requests.get('https://openlibrary.org/trending/forever', verify=False)


c:\Users\HP\Documents\projet\openlib_scarping_with_python\.venv\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'openlibrary.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>

In [8]:
URL = 'https://openlibrary.org/trending/forever'

soup = soup_func(URL)

# pagination list pages
pagination_links = [link['href'] for link in soup.find_all('a', attrs={'class': 'ChoosePage'})][:-1:]
pagination_links = ['/trending/forever?page=1'] + pagination_links
print(pagination_links)

['/trending/forever?page=1', '/trending/forever?page=2', '/trending/forever?page=3', '/trending/forever?page=4', '/trending/forever?page=5', '/trending/forever?page=6', '/trending/forever?page=7', '/trending/forever?page=8', '/trending/forever?page=9', '/trending/forever?page=10']


In [9]:
def jump_data(path:str, data:list):
    import json
    with open(path, 'w') as f:
        json.dump(data, f, indent=4)
        

In [15]:

def scrape(pagination_links):
    # insert into a dictionary format books detail
    data = []
    for pagination_link in pagination_links:
        

        URL = f"https://openlibrary.org{pagination_link}"

        #scarpe the actual pagination link page
        pagination_soup = soup_func(URL)

        # get the list of books of this actual page
        list_group = pagination_soup.find_all('li', attrs={'class': 'searchResultItem'})

        # get the link detail page of every book
        links_detail_book_page = [link.find('a', attrs={'itemprop': 'url'})['href'] for link in list_group]
        for books_detail in links_detail_book_page:
            NEW_URL = f"https://openlibrary.org{books_detail}"
            #book_detail = requests.get(NEW_URL)
            new_soup = soup_func(NEW_URL) #BeautifulSoup(book_detail.text, 'html.parser')
            publishers = [publisher.text for publisher in new_soup.find_all('a', attrs={'itemprop': 'publisher'})]
            try:
                isbn = [isbn.text for isbn in new_soup.find_all('dd', attrs={'class': 'object', 'itemprop': 'isbn'})]
                isbn10 = isbn[0]
                isbn15 = isbn[1]
            except:
                isbn10 = None
                isbn15 = None
            try:
                page = new_soup.find('span', attrs={'class':'edition-pages' ,'itemprop': 'numberOfPages'}).text
            except:
                page = None
            try:
                language = [language.a.text for language in new_soup.find_all('span', attrs={'itemprop':'inLanguage'})]
            except ValueError:
                language = None
            try:
                title = new_soup.find('h1', attrs={'class':'work-title', 'itemprop':'name'}).text
            except:
                title = None
            book_data = {
                    'title': title,
                    'author': new_soup.find('a', attrs={'itemprop': 'author'}).text, #xpath(new_soup, '//*[@id="contentBody"]/div[1]/div[3]/div[2]/span/h2[2]/a'),
                    'publication_date': xpath(new_soup, '//*[@id="contentBody"]/div[1]/div[3]/div[5]/div/div[1]/span'),
                    'page': page,
                    'language': ' '.join(language),
                    'publishers': f"{', '.join(publishers)}",
                    'description': xpath(new_soup, '//*[@id="contentBody"]/div[1]/div[3]/div[4]/div/p[1]'),
                    'isbn10': isbn10, #xpath(new_soup, '//*[@id="contentBody"]/div[1]/div[3]/div[9]/div[4]/dl/dd[4]'),
                    'isbn15': isbn15,
                    'image':new_soup.find('img', attrs={'itemprop': 'image'})['src'],
                    'genre':[a.text for a in new_soup.find_all('a', attrs={'data-ol-link-track' : "BookOverview|SubjectClick"})]
                    }
            # print(book_data)    
            data.append(book_data)
       
    return jump_data('data/books.json', data)

In [16]:
scrape(pagination_links)

SSLError: HTTPSConnectionPool(host='openlibrary.org', port=443): Max retries exceeded with url: /works/OL81613W?edition=ia%3Aesroman0000king_s8m4 (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))

<h1 style='text-align:center'> Scarping With Selenium</h1>

In [14]:
options = webdriver.EdgeOptions()
service = Service(executable_path='webdriver/edgedriver_win64/msedgedriver.exe')
driver = webdriver.Edge(service=service, options=options)
driver.get('https://openlibrary.org/')

wait = WebDriverWait(driver, 50)
# explorer_menu = wait.until(EC.element_to_be_clickable((By.XPATH, '//summary[contains(text(), "Explorer")]')))
driver.find_element(By.XPATH, '//summary[contains(text(), "Explorer")]').click()
# Cliquez sur le menu "Explorer" pour ouvrir le menu déroulant
# explorer_menu.click()

# driver.quit()



<selenium.webdriver.remote.webelement.WebElement (session="8aac889aea3a1dac19e7c3a10005f47f", element="f.8A226494ADCAB991FD9E41AD585CFA88.d.78F037AF99DDCC951C17B65EF1973F62.e.30")>